In [1]:
import pandas as pd
import openpyxl
import xlsxwriter
from datetime import date


In [35]:
input_df = pd.read_excel('../sample_files/Input_SD.xlsx')
input_df

,Targeting type,SKU,CODE,Market,PPC Type,Tactic,Match Type,BRAND,Date,PIC,STT,Budget,Targeting,Bid,Bid strategy
0,CONTEXTUAL VCPM,set 9 - 001,CC-0031-0002,US,SD,T00020,REACH,Chicnchill,20221212,HUY,1,15,"B09ZNJST22,B0BBF1MM2G",2022-05-05,Dynamic bids - down only\n
1,CONTEXTUAL CPC,set 9 - 001,CC-0031-0002,US,SD,T00020,PAGE VISITS or CVR,Chicnchill,20221212,HUY,1,15,"B09ZNJST22,B0BBF1MM2G",2022-05-05,Dynamic bids - down only\n
2,AUDIENCE CPC,set 9 - 001,CC-0031-0002,US,SD,T00030,PAGE VISITS or CVR,Chicnchill,20221212,HUY,1,15,"category=""166164011""",2022-05-05,Dynamic bids - down only\n
3,AUDIENCE VCPM,set 9 - 001,CC-0031-0002,US,SD,T00030,REACH,Chicnchill,20221212,HUY,1,15,"category=""166164011""",2022-05-05,Dynamic bids - down only\n


<h2>Create Excel Form</h2>

In [27]:
def create_single_row(columns:list,cam_id):
    single_row_dictionary = {}
    for item in columns:
        if item == 'Product':
            single_row_dictionary[item] = 'Sponsored Products'
        elif item == 'Operation':
            single_row_dictionary[item] = 'Create'
        elif item == 'Campaign Id':
            single_row_dictionary[item] = cam_id
        elif item == 'State':
            single_row_dictionary[item] = 'enabled'
        else:
            single_row_dictionary[item] = None
    return single_row_dictionary

In [31]:
def createDataFrame_type1(cam_id,budget,sku,bid_list:list,billing_strategy,date,tatic,targeting_list:list):
    columns = ['Product','Entity','Operation','Campaign Id','Ad Group Id','Campaign Name','Ad Group Name','Start Date','End Date','State','Tactic','Budget Type','Budget','Cost Type','Bid Optimization','SKU','ASIN','Ad Group Default Bid','Bid','Targeting Expression']
    df = pd.DataFrame(columns=columns)
    for i in range(0,3):
        df = df.append(create_single_row(columns=columns,cam_id=cam_id),ignore_index=True)
    
    df.iloc[0]['Entity'] = 'Campaign'
    df.iloc[0]['Campaign Name'] = cam_id
    df.iloc[0]['Start Date'] = date
    df.iloc[0]['Tatic'] = tatic
    df.iloc[0]['Budget Type'] = 'daily'
    df.iloc[0]['Budget'] = budget
    df.iloc[0]['Cost Type'] = 'vCPM'
    
    df.iloc[1]['Entity'] = 'Ad group'
    df.iloc[1]['Ad Group Id'] = cam_id
    df.iloc[1]['Ad Group Name'] = cam_id
    df.iloc[1]['Ad Group Name'] = cam_id
    df.iloc[1]['Bid Optimization'] = 'reach'
    df.iloc[1]['Ad Group Default Bid'] = 1
    
    df.iloc[2]['Entity'] = 'Product ad'
    df.iloc[2]['Ad Group Id'] = cam_id
    df.iloc[2]['SKU'] = sku
    
    for i in range(len(targeting_list)):
        temp_row = create_single_row(columns=columns,cam_id=cam_id)
        temp_row['Entity'] = 'Product targeting'
        temp_row['Ad Group Id'] = cam_id
        if len(bid_list)>1:
            if i < len(bid_list):
                temp_row['Bid'] = bid_list[i]
            else:
                temp_row['Bid'] = bid_list[-1]
        else:
            temp_row['Bid'] = bid_list[0]
        if targeting_list[i].startswith("BO"):
            temp_row['Targeting Expression'] = '''asin="''' +str(targeting_list[i])+ '''"'''
        else:
            temp_row['Targeting Expression'] = str(targeting_list[i])
        df = df.append(temp_row,ignore_index=True)
    
    return df

In [34]:
createDataFrame_type1(cam_id='ABC123',budget=1000,sku='CC-100',bid_list=[5.5,6.5],billing_strategy='Bid downly',date='20221212',tatic='T00020',targeting_list=['BOProduct1','BOproduct2'])

C:\Users\dvkhai2\AppData\Local\Temp\ipykernel_4456\1746860950.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(create_single_row(columns=columns,cam_id=cam_id),ignore_index=True)
C:\Users\dvkhai2\AppData\Local\Temp\ipykernel_4456\1746860950.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(create_single_row(columns=columns,cam_id=cam_id),ignore_index=True)
C:\Users\dvkhai2\AppData\Local\Temp\ipykernel_4456\1746860950.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(create_single_row(columns=columns,cam_id=cam_id),ignore_index=True)
C:\Users\dvkhai2\AppData\Local\Temp\ipykernel_4456\1746860950.py:41: FutureWarning: The frame.append method is deprecated and will be removed 

,Product,Entity,Operation,Campaign Id,Ad Group Id,Campaign Name,Ad Group Name,Start Date,End Date,State,Tactic,Budget Type,Budget,Cost Type,Bid Optimization,SKU,ASIN,Ad Group Default Bid,Bid,Targeting Expression
0,Sponsored Products,Campaign,Create,ABC123,None,ABC123,None,20221212,None,enabled,None,daily,1000,vCPM,None,None,None,None,NaN,None
1,Sponsored Products,Ad group,Create,ABC123,ABC123,None,ABC123,None,None,enabled,None,None,None,None,reach,None,None,1,NaN,None
2,Sponsored Products,Product ad,Create,ABC123,ABC123,None,None,None,None,enabled,None,None,None,None,None,CC-100,None,None,NaN,None
3,Sponsored Products,Product targeting,Create,ABC123,ABC123,None,None,None,None,enabled,None,None,None,None,None,None,None,None,5.5,"asin=""BOProduct1"""
4,Sponsored Products,Product targeting,Create,ABC123,ABC123,None,None,None,None,enabled,None,None,None,None,None,None,None,None,6.5,"asin=""BOproduct2"""
